# Song Generator

Idea: use the chord progressions thing to generate sequence, then a melody over that sequence that fits according to the number of beats in an input poem

![](poss_chords_prog.png)

---

### Structure
1. Generate chord sequence
2. Do this for verses, chorus, and bridge
3. Parse input test, find number of syllables per line
4. Generate melody with same number of notes over the chord progression
5. (maybe) export to midi files
6. Put into ableton
7. Record vocals
8. Repeat over and over and over...
9. Profit

### Next steps
- Add arpeggios
- Add bassline
- Add bass harmonies
- Add melody harmonies

### SongGen stuff
---

In [10]:
import random
from midiutil.MidiFile import MIDIFile
from datetime import datetime
import os

In [11]:
class SongGeneration:
    
    def __init__(self, key, style='major', length=4, verses=1, rhythm=[], file_name="test", file_location=""):
        self.key = key
        self.style = style # 'major' or 'minor'
        self.length = length # default is 4 chords
        self.verses = verses # default is 1 verse
        self.file_name = file_name # maximum recommended length: 9
        self.file_location = file_location # where to save the midi file
        
        # Chord Progression Dictionaries
        self.chord_prog_maj = {'I':[['V', 'vii'], ['ii', 'IV'], ['vi'], ['iii']],
                          'ii':['V', 'vii'],
                          'iii': ['vi'],
                          'IV':['V', 'vii'],
                          'V': ['I'],
                          'vi': ['ii', 'IV'],
                          'vii': ['iii', 'I']}
        
        self.chord_prog_min = {'i':[['V'], ['ii', 'iv'], ['VI'], ['III'], ['VII']],
                          'ii':['V'],
                          'III': ['VI'],
                          'iv':['V'],
                          'V': ['i'],
                          'VI': ['ii', 'iv'],
                          'VII' : ['III']}
        
        self.chord_prog = self.chord_prog_maj if self.style == 'major' else self.chord_prog_min
        
        self.one_cycle = 4 if self.style == 'major' else 5 # for use in chord prog picking
        
        # List of all possible notes, rhythms, and rhythm weights
        self.all_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        self.all_rhythms = [0.25, 0.5, 1, 2]
        self.all_rhythm_weights = [0.1, 0.4, 0.95, 1.0]
        
        # Notes dictionary for MIDI file
        self.notes_dict = {}
        bass = 48
        treble1 = 60
        treble2 = 72
        for i in range(0, len(self.all_notes)):
            self.notes_dict[self.all_notes[i]] = [bass + i, treble1 + i, treble2 + i]
        
        # Class variables, for use in generation functions
        self.scale = []
        self.chords = []
        self.chord_notes = []
        self.melody = []
        self.rhythm_intro = rhythm
        self.rhythm = []
    
    def __str__(self):
        return_string = str(self.key) + " " + str(self.style) + "\n" + str(self.chords) + "\n" 
        #return_string += str(self.chord_notes) + "\n" + str(self.rhythm) + "\n"
        #return_string += str(self.melody)
        return return_string
    
    def gen_song(self):
        self.gen_scale()
        self.gen_chords()
        for i in range(self.verses):
            self.gen_rhythm()
            self.gen_melody()
        self.gen_MIDI()
        
    def gen_scale(self):
        i = self.all_notes.index(self.key)
        l = len(self.all_notes)
        if self.style == 'major':
            self.scale.append(self.all_notes[i])
            self.scale.append(self.all_notes[(i + 2) % l])
            self.scale.append(self.all_notes[(i + 4) % l])
            self.scale.append(self.all_notes[(i + 5) % l])
            self.scale.append(self.all_notes[(i + 7) % l])
            self.scale.append(self.all_notes[(i + 9) % l])
            self.scale.append(self.all_notes[(i + 11) % l])
        elif self.style == 'minor':
            self.scale.append(self.all_notes[i])
            self.scale.append(self.all_notes[(i + 2) % l])
            self.scale.append(self.all_notes[(i + 3) % l])
            self.scale.append(self.all_notes[(i + 5) % l])
            self.scale.append(self.all_notes[(i + 7) % l])
            self.scale.append(self.all_notes[(i + 8) % l])
            self.scale.append(self.all_notes[(i + 10) % l])
        
    def gen_chords(self):
        chords = ['I'] if self.style == 'major' else ['i']
        sub_value = 2
        start_value = (self.length - sub_value) % self.one_cycle
        start_chord = random.choice(self.chord_prog[chords[0]][start_value])
        chords.append(start_chord)
        
        while len(chords) != self.length:
            if chords[-1] != 'I' and chords[-1] != 'i':
                next_chord = random.choice(self.chord_prog[chords[-1]])  
            elif chords[-1] == 'iv' and len(chords) >= self.length - 2:
                next_chord = 'V'
            else:
                if self.style == 'major':
                    new_start_value = (self.length - sub_value) % self.one_cycle
                else: 
                    new_start_value = (self.length - 5) % self.one_cycle
                next_chord = random.choice(self.chord_prog[chords[-1]][new_start_value])
            chords.append(next_chord)
            
        self.chords = chords
        
        for chord in self.chords:
            if chord == 'I' or chord == 'i':
                self.chord_notes.append([self.scale[0], self.scale[2], self.scale[4]])
            elif chord == 'ii':
                self.chord_notes.append([self.scale[1], self.scale[3], self.scale[5]])
            elif chord == 'iii' or chord == 'III':
                self.chord_notes.append([self.scale[2], self.scale[4], self.scale[6]])
            elif chord == 'IV' or chord == 'iv':
                self.chord_notes.append([self.scale[3], self.scale[5], self.scale[0]])
            elif chord == 'V':
                self.chord_notes.append([self.scale[4], self.scale[6], self.scale[1]])
            elif chord == 'vi' or chord == 'VI':
                self.chord_notes.append([self.scale[5], self.scale[0], self.scale[2]])
            elif chord == 'vii' or chord == 'VII':
                self.chord_notes.append([self.scale[6], self.scale[1], self.scale[3]])
        #print(self.chords)
    
    def gen_rhythm(self):
        # probability of measures 1/3 and 2/4 being identical in a 4 bar series
        prob_1_3_same = 0.5
        prob_2_4_same = 0.9
        
        if len(self.rhythm_intro) == 0:
            for i in range(len(self.chords)):
                # chance of skipping generation and repeating the rhythm
                if i > 1 and len(self.chords) == 4:
                    r_same = random.uniform(0, 1)
                    if (i == 2 and r_same < prob_1_3_same) or (i == 3 and r_same < prob_2_4_same):
                        self.rhythm.append(self.rhythm[i-2])
                        continue
                    
                chord = self.chords[i]
                total = 0
                measure = []
                while total < 4:
                    # pick a weighted random note
                    r1 = random.uniform(0, 1)
                    new_note = self.all_rhythms[-1]  
                    for i in range(0, len(self.all_rhythm_weights)):
                        if self.all_rhythm_weights[i] <= r1:
                            new_note = self.all_rhythms[i]                            
                    
                    # only add note size that fits inside the measure
                    if total + new_note > 4:
                        new_note = 4 - total
                    measure.append(new_note)
                    total += new_note
                    
                    # some percent chance to replicate the same note, creating runs of the same size
                    # only if the note is < one beat long
                    if new_note < 1 and total + new_note <= 4:
                        r2 = random.uniform(0, 1)
                        if r2 < 0.5:
                            measure.append(new_note)
                            total += new_note
                    
                self.rhythm.append(measure)
                
        else:
            for i in range(0, len(self.chords)):
                # chance of skipping generation and repeating the rhythm
                if i > 1 and len(self.chords) == 4:
                    r_same = random.uniform(0, 1)
                    if (i == 2 and r_same < prob_1_3_same) or (i == 3 and r_same < prob_2_4_same):
                        self.rhythm.append(self.rhythm[i-2])
                        continue
                
                num_notes = self.rhythm_intro[i]
                total = 4
                rough_notes = total / num_notes
                raw_measure = [rough_notes for i in range(0, num_notes)]
                measure = []
                for note in raw_measure:
                    #self.all_rhythms = [0.25, 0.5, 1, 2]
                    closest_two = [0.25, 0.5]
                    if note > 0.5:
                        closest_two[0] = 1
                    if note > 1:
                        closest_two[1] = 2
                        
                    measure.append(random.choice(closest_two))
                
                if sum(measure) > 4:
                    while sum(measure) > 4:
                        max_note = max(measure)
                        note_loc = measure.index(max_note)
                        if max_note > sum(measure) - 4:
                            new_note = max_note - (sum(measure) - 4)
                        else:
                            new_note = max_note / 2
                        measure[note_loc] = new_note
                elif sum(measure) < 4:
                    while sum(measure) < 4:
                        min_note = min(measure)
                        note_loc = measure.index(min_note)
                        new_note = min_note + (4 - sum(measure))
                        measure[note_loc] = new_note
                        
                self.rhythm.append(measure)
                        
                
    
    def gen_melody(self):
        # probability of measures 1/3 and 2/4 being identical in a 4 bar series
        prob_1_3_same = 0.5
        prob_2_4_same = 0.9
        
        for i in range(0, len(self.chords)):
            # chance of skipping generation and repeating the melody
            if i > 1 and self.rhythm[i] == self.rhythm[i-2] and len(self.chords) == 4:
                r_same = random.uniform(0, 1)
                if (i == 2 and r_same < prob_1_3_same) or (i == 3 and r_same < prob_2_4_same):
                    self.melody.append(self.melody[i-2])
                    continue
                
            starting_note = random.choice(self.chord_notes[i]) 
            measure = [[starting_note, self.rhythm[i][0]]]
            for j in range(1, len(self.rhythm[i])):
                prev_note = measure[j-1][0]
                scale_i = self.scale.index(prev_note)
                if self.rhythm[i][j] < 2:
                    if scale_i + 1 == len(self.scale):
                        note = random.choice([self.scale[scale_i - 1], self.scale[scale_i]])
                    elif scale_i - 1 == -1:
                        note = random.choice([self.scale[scale_i + 1], self.scale[scale_i]])
                    else:
                        note = random.choice([self.scale[scale_i + 1], 
                                              self.scale[scale_i - 1], 
                                              self.scale[scale_i]])
                else:
                    for i in range(scale_i):
                        note_below = self.scale[max(scale_i - i, 0)]
                        note_above = self.scale[min(scale_i + i, len(self.scale) - 1)]
                        if note_below in self.chord_notes:
                            note = note_below
                        elif note_above in self.chord_notes:
                            note = note_above
                measure.append([note, self.rhythm[i][j]])
            self.melody.append(measure)
    
    def gen_MIDI(self):
        #print(str(self.melody))
        # create your MIDI object
        mf = MIDIFile(2)
        track1 = 0
        track2 = 1

        time = 0
        mf.addTrackName(track1, time, self.file_name + "_chords")
        mf.addTrackName(track2, time, self.file_name + "_melody")
        #mf.addTempo(track1, time, 120)

        # add some notes
        channel = 0
        channel_chords = 0
        channel_melody = 1
        volume = 100
        
        overall_chord_time = 0
        for v in range(self.verses):
            for i in range(0, len(self.chord_notes)):
                chord = self.chord_notes[i]
                chord_time = overall_chord_time + i * 4
                pitch1 = self.notes_dict[chord[0]][0]
                pitch2 = self.notes_dict[chord[1]][0]
                pitch3 = self.notes_dict[chord[2]][0]
                duration = 4
                mf.addNote(track1, channel, pitch1, chord_time, duration, 65)
                mf.addNote(track1, channel, pitch2, chord_time, duration, 65)
                mf.addNote(track1, channel, pitch3, chord_time, duration, 65)
            overall_chord_time += len(self.chord_notes) * 4
        
        last_pitch = 0
        for i in range(0, len(self.melody)):
            measure = self.melody[i]
            measure_time = i * 4
            
            for j in range(0, len(measure)):
                note = measure[j][0]
                pitch1 = self.notes_dict[note][1]
                pitch2 = self.notes_dict[note][2]
                
                dist1 = (pitch1 - last_pitch)**2
                dist2 = (pitch2 - last_pitch)**2
                
                pitch = pitch1 if dist1 <= dist2 else pitch2                
                duration = measure[j][1]
                mf.addNote(track2, channel, pitch, measure_time, duration, volume)
                
                measure_time += measure[j][1]
                last_pitch = pitch
            

        # write it to disk
        with open("midi_files\\" + self.file_location + self.file_name + ".mid", 'wb') as outf:
            mf.writeFile(outf)

In [12]:
note_list = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
key = random.choice(note_list)
print(key)

D


In [13]:
test_song = SongGeneration(key, style='minor', rhythm = [7, 8, 7, 8], length=4, verses=1)
test_song.gen_song()
print(test_song.scale)

['D', 'E', 'F', 'G', 'A', 'A#', 'C']


In [14]:
folder = "test_august_2021\\"
song_style = "minor"
song_info_file = "song_info.txt"
# my personal script directory
script_dir = "D:\\Documents\\Github\\musicGenerator\\midi_files\\"
f = open(script_dir + folder + song_info_file, "w")

verse1 = SongGeneration(key, style=song_style, length=4, verses=2, file_name="verse1", file_location=folder)
verse1.gen_song()
f.write("Verse 1\n")
f.write(str(verse1))

chorus = SongGeneration(key, style=song_style, length=4, verses=2, file_name="chorus", file_location=folder)
chorus.gen_song()
f.write("\nChorus\n")
f.write(str(chorus))

verse2 = SongGeneration(key, style=song_style, length=4, verses=2, file_name="verse2", file_location=folder)
verse2.gen_song()
f.write("\nVerse 2\n")
f.write(str(verse2))

bridge = SongGeneration(key, style=song_style, length=7, verses=1, file_name="bridge", file_location=folder)
bridge.gen_song()
f.write("\nBridge\n")
f.write(str(bridge))

f.close()

In [ ]:
# now = str(datetime.now()) # current date and time
# now = now.replace(":", "")
# now = now.replace(" ", "_")
# now = now.replace(".", "_")

### Natural Language Processing

NLTK specifically

---

In [ ]:
import nltk
from nltk.corpus import cmudict
from nltk.tokenize import RegexpTokenizer
import math
d = cmudict.dict()

In [ ]:
def nsyl(word):
    vowels = 'aeiouy'
    if len(word) == 1 and word.lower()[0] not in vowels:
        return [0]
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]
    except KeyError:
        #if word not found in cmudict
        return [syllables(word)]

def syllables(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    return count

In [ ]:
f = open("poems\\places_winter_sun_sara_teasdale.txt")
raw_sonnets = f.readlines()
f.close()
print(raw_sonnets)

In [ ]:
sonnet = []
for line in raw_sonnets:
    new_line = line[:-1]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(new_line)
    if len(tokens) > 0:
        sonnet.append(tokens)
print(sonnet)

In [ ]:
syllable_list = []
for line in sonnet:
    new_line = []
    for word in line:
        new_line.append([word.lower(), nsyl(word)])
    syllable_list.append(new_line)
print(syllable_list)

In [ ]:
counts = []
lyrics = []
for line in syllable_list:
    count = 0
    sentence = []
    for word in line:
        #sentence += word[0] + " "
        count += max(word[1])
    counts.append(count)
    lyrics.append(sentence)
    
    next_count = 0
    sentence.append("")
    for word in line:
        next_count += max(word[1])
        if next_count > math.ceil(count/2) and len(sentence) == 1:
            sentence.append("")
        sentence[-1] += word[0] + " "
        
print(counts)
print(lyrics)

In [ ]:
folder = "album_2\\track_1\\"
song_style = "major"
song_info_file = "song_info.txt"
# my personal script directory
script_dir = "D:\\Documents\\Github\\musicGenerator\\midi_files\\"
f = open(script_dir + folder + song_info_file, "w")

verse1_lyrics = """Whose woods these are I think I know.
His house is in the village though;
He will not see me stopping here
To watch his woods fill up with snow.

My little horse must think it queer
To stop without a farmhouse near
Between the woods and frozen lake
The darkest evening of the year."""
verse1_rhythm = [8, 8, 8, 8]
verse1 = SongGeneration(key, style=song_style, rhythm = verse1_rhythm, length=4, verses=2, file_name="verse1", file_location=folder)
verse1.gen_song()
f.write("Verse 1\n")
f.write("Lyrics:\n" + verse1_lyrics + "\n")
f.write(str(verse1))

chorus_lyrics = """The night is darkening round me,
The wild winds coldly blow;
But a tyrant spell has bound me
And I cannot, cannot go.

The giant trees are bending
Their bare boughs weighed with snow.
And the storm is fast descending,
And yet I cannot go.

Clouds beyond clouds above me,
Wastes beyond wastes below;
But nothing drear can move me;
I will not, cannot go."""
chorus_rhythm = [4, 4, 4, 4]
chorus = SongGeneration(key, style=song_style, rhythm = chorus_rhythm, length=4, verses=2, file_name="chorus", file_location=folder)
chorus.gen_song()
f.write("\nChorus\n")
f.write("Lyrics (1 stanza each time, twice):\n" + chorus_lyrics + "\n")
f.write(str(chorus))

verse2_lyrics = """He gives his harness bells a shake
To ask if there is some mistake.
The only other sound’s the sweep
Of easy wind and downy flake.

The woods are lovely, dark and deep,
But I have promises to keep,
And miles to go before I sleep,
And miles to go before I sleep."""
verse2_rhythm = [8, 8, 9, 9]
verse2 = SongGeneration(key, style=song_style, rhythm = verse2_rhythm, length=4, verses=2, file_name="verse2", file_location=folder)
verse2.gen_song()
f.write("\nVerse 2\n")
f.write("Lyrics:\n" + verse2_lyrics + "\n")
f.write(str(verse2))

bridge_lyrics = """There was a bush with scarlet berries,
    And there were hemlocks heaped with snow,
With a sound like surf on long sea-beaches
    They took the wind and let it go.

The hills were shining in their samite,
    Fold after fold they flowed away;
"Let come what may," your eyes were saying,
    "At least we two have had to-day." """
bridge_rhythm = [9, 8, 10, 8, 9, 8, 9, 8]
bridge = SongGeneration(key, style=song_style, length=7, verses=1, file_name="bridge", file_location=folder)
bridge.gen_song()
f.write("\nBridge\n")
f.write("Lyrics:\n" + bridge_lyrics + "\n")
f.write(str(bridge))

f.close()

In [ ]:
counts_split = []
for i in range(0, len(counts)-1):
    count1 = counts[i]
    count2 = counts[i+1]
    count3 = [math.ceil(count1/2), math.floor(count1/2), math.ceil(count2/2), math.floor(count2/2)]
    counts_split.append(count3)
print(counts_split)

In [ ]:
# stanza_num = 1
# for stanza in counts_split:
#     count_song = SongGeneration('G', length=len(stanza), rhythm=stanza, file_name="stanza_"+str(stanza_num))
#     count_song.gen_song()
#     print("Generated stanza " + str(stanza_num))
#     stanza_num += 1